In [1]:
import numpy as np # scientific computing
import pandas as pd # data loading and processing
import os # os operations
import matplotlib.pyplot as plt # for generating figures
import math
import matplotlib.dates as mdates
import seaborn as sns # for generating visualizations, better support with pandas than matplotlib
from scipy import stats

In [2]:
def get_gene_names(filename,col=None):
    file = pd.read_csv(filename, index_col=None, header= 0).T
    names = file[col].dropna().tolist()
    return names

def construct_filename(c, db):
    if db =="xena":
        n1 = "./data/" + "TCGA." + c + ".sampleMap_HiSeqV2"
        n2 = "./data/" + "TCGA." + c + ".sampleMap_"+ c +"_clinicalMatrix"
    elif db == "cbio":
        n1 = "./data/" + c+"_data_mrna_seq_v2_rsem.txt"
        n2 = "./data/" + c + "_data_clinical_sample.txt"
    else:
        print("db must be either xena or cbio")
    return n1,n2

def construct_hccdb_filename(n):
    n1 = "./data/HCCDB/HCCDB" + n + "_mRNA_level3.txt"
    n2 = "./data/HCCDB/HCCDB" + n  + ".sample.txt"
    return n1,n2

def process_data(df, targets, y_var_names, pheno_filtered=None):

    # subset to get relevant genes
    df_filtered = df.loc[targets]
    # print(df_filtered)
    print(df_filtered.isnull().values.any())

    # check for nan values
    na_filter = df_filtered.isnull().any()
    na_ls = na_filter[na_filter == True].index.to_list()
    print(na_ls)
    
    # impute nan values
    mean_ls = df_filtered.mean()
    for na_col in na_ls:
        df_filtered[na_col] = df_filtered[na_col].fillna(mean_ls[na_col])
    
    print(df_filtered.isnull().values.any())
    
    df_filtered = df_filtered.T # patients x genes
    df_filtered = df_filtered.astype(np.float64)

    # scale numerical data
    df_filtered = np.log10(df_filtered+1)

    # for each sequenced gene were rescaled to set the median equal to 1
    df_filtered=((df_filtered-df_filtered.median())/df_filtered.std())+1

    data = df_filtered   

    # take only nrf2 target genes
    y_var_gene_set = data[y_var_names]
    data.drop(y_var_names, inplace = True, axis = 1)
    y_var_gene_set["composite_score"] = y_var_gene_set.mean(axis = 1)
    data = pd.concat([data, y_var_gene_set], axis = 1) # patients x genes 
    
    return data

def analyse(data, fig, name, ax, fn, x_label, y_label, x_target = "RRM2B", y_target = "composite_score", ):
    #find line of best fit
    y, x = data[y_target].to_numpy(), data[x_target].to_numpy()
    a, b = np.polyfit(x, y, 1)

    iqr = data[x_target].T.describe()

    # bin the patients into quartiles based on G6PD expression
    data["RRM2B levels"] = pd.cut(data["RRM2B"],
                    bins=[ iqr["min"], iqr["25%"], iqr["75%"], iqr["max"]],
                    labels=["Bottom 25%", "-", "Top 25%"])

    # get r sq val
    r = np.corrcoef(x, y)[0, 1]

    #find p-value
    n = data.shape[0]
    t = (r-math.sqrt(n-2))/math.sqrt(1-(r**2))
    p = stats.t.sf(abs(t), df=n)*2

    # plot the data
    # scatter plot for RRM2B against NRF2 activity
    sns.set_style("whitegrid")
    sns.set()
    sns.scatterplot(data=data, x=x_target, y=y_target, ax= ax)
    ax.plot(x, a*x+b, color="black")
    ax.set_ylabel(y_label,fontsize = 28)
    ax.set_xlabel(x_label + " \n (r = " + str(round(r, 4)) + "," + " p = " + str(round(p, 4)) +")",fontsize = 25)
    ax.set_title(name, fontsize = 30)
    ax.tick_params(axis='both', which='major', labelsize=25)
    plt.show()

    # save the figure 
    fig.savefig(fn)

def get_targets_present(data, targets):
    idx = data.index.to_list()
    # print(idx)
    # print(targets)
    targets_present = list(set(idx).intersection(set(targets)))
    # print(targets_present)
    return targets_present

def get_xena_data(n1):
    df = pd.read_csv(n1, index_col = 0, sep = "\t") # gene x patient
    return df

def get_cbio_data(n1):
    df = pd.read_csv(n1, index_col = 0, sep = "\t").drop(["Entrez_Gene_Id"], axis=1) # gene x patient
    return df

def get_hccdb_data(n1):
    df = pd.read_csv(n1, index_col = 1, sep = "\t").drop(["Entrez_ID"], axis=1) # gene x patient
    return df

def get_xena_pheno(n2):
    pheno = pd.read_csv(n2, index_col=0, sep = "\t")
    pheno = pheno[["sample_type"]]
    pheno_filtered = pheno.dropna()
    return pheno_filtered

def get_cbio_pheno(n2):
    pheno = pd.read_csv(n2, index_col=1,header = 4, sep = "\t")
    pheno = pheno[["SAMPLE_TYPE"]]
    pheno_filtered = pheno.dropna()
    return pheno_filtered

def get_hccdb_pheno(n2):
    pheno = pd.read_csv(n2, index_col=0, sep = "\t").T
    pheno = pheno[["TYPE"]]
    pheno_filtered = pheno.dropna()
    return pheno_filtered



In [3]:
# script to consolidate all HCCDB data into one dataframe

hccdb_names = ["1", "3", "4",  "8", "9", "11", "12", "13", "14", "16", "17", "18"]
hccdb = pd.DataFrame()

for i in range(len(hccdb_names)):
    n1, n2 = construct_hccdb_filename(hccdb_names[i])
    hccdb_temp = get_hccdb_data(n1)
    hccdb_temp = hccdb_temp.loc[~hccdb_temp.index.duplicated(),:].copy()
    hccdb_temp.loc["ptype",:] = "HCCDB-" + hccdb_names[i]
    hccdb = pd.concat([hccdb, hccdb_temp], axis = 1) # patients x genes
    


/var/folders/mp/lfr86vf57gzgqs4tr6qd5bs80000gn/T/ipykernel_46037/89867001.py:112: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(n1, index_col = 1, sep = "\t").drop(["Entrez_ID"], axis=1) # gene x patient


In [4]:
# load pancan data

tcga = pd.read_csv("./data/EB++AdjustPANCAN_IlluminaHiSeq_RNASeqV2.geneExp (1).xena", index_col = 0, sep = "\t") # gene x patient
pheno = pd.read_csv("./data/TCGA_phenotype_denseDataOnlyDownload (1).tsv", index_col = 0, sep = "\t") # patient x phenotype

In [ ]:
# attach cancer type to each patient
data = tcga.T
data = pd.concat([data, pheno], axis = 1, join = "inner") # patients x genes

In [ ]:
print(data.shape)
print(tcga.T.shape)

In [ ]:
# attach abbeviations for each cancer type
ls = data["_primary_disease"].unique().tolist()

conditions = [
    data['_primary_disease'] == 'adrenocortical cancer',
    data['_primary_disease'] == 'bladder urothelial carcinoma',
    data['_primary_disease'] == 'breast invasive carcinoma',
    data['_primary_disease'] == 'cervical & endocervical cancer',
    data['_primary_disease'] == 'cholangiocarcinoma', 
    data['_primary_disease'] == 'colon adenocarcinoma',
    data['_primary_disease'] == 'diffuse large B-cell lymphoma',
    data['_primary_disease'] == 'esophageal carcinoma',
    data['_primary_disease'] == 'glioblastoma multiforme',
    data['_primary_disease'] == 'head & neck squamous cell carcinoma',
    data['_primary_disease'] == 'kidney chromophobe',
    data['_primary_disease'] == 'kidney clear cell carcinoma',
    data['_primary_disease'] == 'kidney papillary cell carcinoma',
    data['_primary_disease'] == 'acute myeloid leukemia',
    data['_primary_disease'] == 'brain lower grade glioma',
    data['_primary_disease'] == 'liver hepatocellular carcinoma',
    data['_primary_disease'] == 'lung adenocarcinoma',
    data['_primary_disease'] == 'lung squamous cell carcinoma',
    data['_primary_disease'] == 'mesothelioma',
    data['_primary_disease'] == 'ovarian serous cystadenocarcinoma',
    data['_primary_disease'] == 'pancreatic adenocarcinoma',
    data['_primary_disease'] == 'pheochromocytoma & paraganglioma',
    data['_primary_disease'] == 'prostate adenocarcinoma',
    data['_primary_disease'] == 'rectum adenocarcinoma',
    data['_primary_disease'] == 'sarcoma',
    data['_primary_disease'] == 'skin cutaneous melanoma',
    data['_primary_disease'] == 'stomach adenocarcinoma',
    data['_primary_disease'] == 'testicular germ cell tumor',
    data['_primary_disease'] == 'thyroid carcinoma',
    data['_primary_disease'] == 'thymoma',
    data['_primary_disease'] == 'uterine corpus endometrioid carcinoma',
    data['_primary_disease'] == 'uterine carcinosarcoma',
    data['_primary_disease'] == 'uveal melanoma'    
]

choices = ["ACC",
           "BLCA",
           "BRCA",
           "CESC",
           "CHOL",
           "COAD",
           "DBLC",
           "ESCA",
           "GBM",
           "HNSC",
           "KICH",
           "KIRC",
           "KIRP",
           "LAML",
           "LGG",
           "LIHC",
           "LUAD",
           "LUSC",
           "MESO",
           "OV",
           "PAAD",
           "PCPG",
           "PRAD",
           "READ",
           "SARC",
           "SKCM",
           "STAD",
           "TGCT",
           "THCA",
           "THYM",
           "UCEC",
           "UCS",
           "UVM"
           ]

data["ptype"] = np.select(conditions, choices, default = "null")
data.head()

In [ ]:
# # master script to screen cancers for RRM2B expression vs antioxidant signature

# gene_set = pd.read_csv("./data/oxstress genes.csv", index_col=None, header= 0)
# gene_set = gene_set['Review v3'].dropna().tolist()

# targets = list(set(["G6PD", "RRM2B"] +  gene_set))

# rval1 =[]
# rval2 = []
# databases = ['HCCDB-1', 'HCCDB-3', 'HCCDB-4',  'HCCDB-8', 'HCCDB-9', 'HCCDB-11', 
#        'HCCDB-12', 'HCCDB-13', 'HCCDB-14', 'HCCDB-16', 'HCCDB-17', 'HCCDB-18',
#        'ACC', 'BLCA', 'DBLC', 'UCEC', 'SKCM', 'HNSC', 'PRAD', 'KIRP',
#        'PAAD', 'SARC', 'CESC', 'COAD', 'LUSC', 'READ', 'KIRC', 'LIHC',
#        'BRCA', 'OV', 'UCS', 'GBM', 'KICH', 'THCA', 'LGG', 'LUAD', 'MESO',
#        'PCPG', 'TGCT', 'UVM', 'THYM', 'CHOL', 'ESCA', 'STAD', 'LAML','PANCAN'] # , , 'PANCAN'

# # define subplot grid
# fig, axs = plt.subplots(6, 8, figsize=(60, 40), sharey=True)
# plt.subplots_adjust(hspace=0.6)
# fig.suptitle('RRM2B vs antioxidant signature',fontsize = 40)

# for db, ax in zip(databases, axs.ravel()):
#     print(db)
#     if db.startswith("HCCDB"):
#         df = hccdb.T
#         df = df[df["ptype"] == db]
#         df = df.T
#         df.drop(["ptype"], inplace = True)
#     elif db == "PANCAN":
#         df = data
#         df = df.T
#         df.drop(["ptype","sample_type_id", "sample_type", "_primary_disease"], inplace = True)
#     else:
#         df = data[data["ptype"] == db]
#         df = df.T
#         df.drop(["ptype","sample_type_id", "sample_type", "_primary_disease"], inplace = True)	


#     data_new = process_data(df, targets, gene_set, pheno_filtered=None)
#     analyse(data_new, fig, db + " (n = " + str(df.shape[1]) + ")", ax, 'RRM2B vs antioxidant screen.png', x_label = "RRM2B expression", y_label = "Antioxidant signature", x_target = 'RRM2B', y_target = 'composite_score', )
# print("done")


In [ ]:
# # get names of ox_stress target genes

# gene_set = pd.read_csv("./data/oxstress genes.csv", index_col=None, header= 0)
# gene_set = gene_set['NRF2 v3'].dropna().tolist()

# targets = list(set(["G6PD", "RRM2B"] +  gene_set))

# rval1 =[]
# rval2 = []
# cancer = ['ACC', 'BLCA', 'DBLC', 'UCEC', 'SKCM', 'HNSC', 'PRAD', 'KIRP',
#        'PAAD', 'SARC', 'CESC', 'COAD', 'LUSC', 'READ', 'KIRC', 'LIHC',
#        'BRCA', 'OV', 'UCS', 'GBM', 'KICH', 'THCA', 'LGG', 'LUAD', 'MESO',
#        'PCPG', 'TGCT', 'UVM', 'THYM', 'CHOL', 'ESCA', 'STAD', 'LAML','PANCAN'] # , , "PANCAN"

# fig, axs = plt.subplots(1, len(cancer), figsize=(30, 10), sharey=True)
# fig.suptitle('RRM2B vs NRF2 activity',fontsize = 25)

# for c in cancer:
#     n1, n2 = construct_filename(c, "cbio")
#     df = get_cbio_data(n1)
#     pheno_filtered = get_cbio_pheno(n2)
#     data = process_data(df, targets, gene_set, pheno_filtered)
#     analyse(data, fig, axs, cancer, c, "RRM2B vs NRF2 final.png", x_target = "RRM2B", y_target = "composite_score")


In [ ]:
# # master script to screen cancers for RRM2B expression vs NRF2 activity

# gene_set = pd.read_csv("./data/oxstress genes.csv", index_col=None, header= 0)
# gene_set = gene_set['NRF2 v3'].dropna().tolist()

# targets = list(set(["G6PD", "RRM2B"] +  gene_set))

# rval1 =[]
# rval2 = []
# databases = ['HCCDB-1', 'HCCDB-3', 'HCCDB-4',  'HCCDB-8', 'HCCDB-9', 'HCCDB-11', 
#        'HCCDB-12', 'HCCDB-13', 'HCCDB-14', 'HCCDB-16', 'HCCDB-17', 'HCCDB-18',
#        'ACC', 'BLCA', 'DBLC', 'UCEC', 'SKCM', 'HNSC', 'PRAD', 'KIRP',
#        'PAAD', 'SARC', 'CESC', 'COAD', 'LUSC', 'READ', 'KIRC', 'LIHC',
#        'BRCA', 'OV', 'UCS', 'GBM', 'KICH', 'THCA', 'LGG', 'LUAD', 'MESO',
#        'PCPG', 'TGCT', 'UVM', 'THYM', 'CHOL', 'ESCA', 'STAD', 'LAML','PANCAN'] # , , 'PANCAN'

# # define subplot grid
# fig, axs = plt.subplots(6, 8, figsize=(60, 40), sharey=True)
# plt.subplots_adjust(hspace=0.6)
# fig.suptitle('RRM2B vs NRF2 activity',fontsize = 25)

# for db, ax in zip(databases, axs.ravel()):
#     print(db)
#     if db.startswith("HCCDB"):
#         df = hccdb.T
#         df = df[df["ptype"] == db]
#         df = df.T
#         df.drop(["ptype"], inplace = True)
#     elif db == "PANCAN":
#         df = data
#         df = df.T
#         df.drop(["ptype","sample_type_id", "sample_type", "_primary_disease"], inplace = True)
#     else:
#         df = data[data["ptype"] == db]
#         df = df.T
#         df.drop(["ptype","sample_type_id", "sample_type", "_primary_disease"], inplace = True)	


#     data_new = process_data(df, targets, gene_set, pheno_filtered=None)
#     analyse(data_new, fig, db + " (n = " + str(df.shape[1]) + ")", ax, 'RRM2B vs NRF2 screen.png', x_label = "RRM2B expression", y_label = "NRF2 activity", x_target = 'RRM2B', y_target = 'composite_score', )
# print("done")


In [ ]:
# # master script to screen cancers for RRM2B expression vs G6PD

# targets = ["G6PD", "RRM2B"]

# rval1 =[]
# rval2 = []
# databases = ['HCCDB-1', 'HCCDB-3', 'HCCDB-4',  'HCCDB-8', 'HCCDB-9', 'HCCDB-11', 
#        'HCCDB-12', 'HCCDB-13', 'HCCDB-14', 'HCCDB-16', 'HCCDB-17', 'HCCDB-18',
#        'ACC', 'BLCA', 'DBLC', 'UCEC', 'SKCM', 'HNSC', 'PRAD', 'KIRP',
#        'PAAD', 'SARC', 'CESC', 'COAD', 'LUSC', 'READ', 'KIRC', 'LIHC',
#        'BRCA', 'OV', 'UCS', 'GBM', 'KICH', 'THCA', 'LGG', 'LUAD', 'MESO',
#        'PCPG', 'TGCT', 'UVM', 'THYM', 'CHOL', 'ESCA', 'STAD', 'LAML','PANCAN'] # , , 'PANCAN'

# # define subplot grid
# fig, axs = plt.subplots(6, 8, figsize=(60, 40), sharey=True)
# plt.subplots_adjust(hspace=0.6)
# fig.suptitle('RRM2B vs G6PD',fontsize = 40)

# for db, ax in zip(databases, axs.ravel()):
#     print(db)
#     if db.startswith("HCCDB"):
#         df = hccdb.T
#         df = df[df["ptype"] == db]
#         df = df.T
#         df.drop(["ptype"], inplace = True)
#     elif db == "PANCAN":
#         df = data
#         df = df.T
#         df.drop(["ptype","sample_type_id", "sample_type", "_primary_disease"], inplace = True)
#     else:
#         df = data[data["ptype"] == db]
#         df = df.T
#         df.drop(["ptype","sample_type_id", "sample_type", "_primary_disease"], inplace = True)	


#     data_new = process_data(df, targets, ["RRM2B"], pheno_filtered=None)
#     analyse(data_new, fig, db + " (n = " + str(df.shape[1]) + ")", ax, 'RRM2B vs G6PD.png', x_label = "RRM2B", y_label = "G6PD", x_target = 'RRM2B', y_target = 'G6PD', )
# print("done")


In [ ]:
# master script to screen cancers for RRM2B expression vs NRF2 activity

gene_set = pd.read_csv("./data/oxstress genes.csv", index_col=None, header= 0)
gene_set = gene_set['NRF2 v3'].dropna().tolist()

targets = list(set(["G6PD", "RRM2B"] +  gene_set))

rval1 =[]
rval2 = []
databases = ['PANCAN'] # , , 'PANCAN'

# define subplot grid
fig, axs = plt.subplots(6, 8, figsize=(60, 40), sharey=True)
plt.subplots_adjust(hspace=0.6)
fig.suptitle('RRM2B vs antioxidant signature',fontsize = 40)

print("run")
for db, ax in zip(databases, axs.ravel()):
    print(db)
    if db.startswith("HCCDB"):
        df = hccdb.T
        df = df[df["ptype"] == db]
        df = df.T
        df.drop(["ptype"], inplace = True)
    elif db == "PANCAN":
        df = data
        df = df.T
        df.drop(["ptype","sample_type_id", "sample_type", "_primary_disease"], inplace = True)
    else:
        df = data[data["ptype"] == db]
        df = df.T
        df.drop(["ptype","sample_type_id", "sample_type", "_primary_disease"], inplace = True)	


    data_new = process_data(df, targets, gene_set, pheno_filtered=None)
    print(data_new)
    analyse(data_new, fig, db + " (n = " + str(df.shape[1]) + ")", ax, 'RRM2B vs antioxidant screen (PANCAN outlier corrected).png', x_label = "RRM2B expression", y_label = "Antioxidant signature", x_target = 'RRM2B', y_target = 'composite_score', )
print("done")

# # define subplot grid
# fig, axs = plt.subplots(6, 8, figsize=(60, 40), sharey=True)
# plt.subplots_adjust(hspace=0.6)
# fig.suptitle('RRM2B vs NRF2 activity',fontsize = 25)

# for db, ax in zip(databases, axs.ravel()):
#     print(db)
#     if db.startswith("HCCDB"):
#         df = hccdb.T
#         df = df[df["ptype"] == db]
#         df = df.T
#         df.drop(["ptype"], inplace = True)
#     elif db == "PANCAN":
#         df = data
#         df = df.T
#         df.drop(["ptype","sample_type_id", "sample_type", "_primary_disease"], inplace = True)
#     else:
#         df = data[data["ptype"] == db]
#         df = df.T
#         df.drop(["ptype","sample_type_id", "sample_type", "_primary_disease"], inplace = True)	


#     data_new = process_data(df, targets, gene_set, pheno_filtered=None)
#     print(data_new)
#     analyse(data_new, fig, db + " (n = " + str(df.shape[1]) + ")", ax, 'RRM2B vs NRF2 screen (PANCAN outlier corrected).png', x_label = "RRM2B expression", y_label = "NRF2 activity", x_target = 'RRM2B', y_target = 'composite_score', )
# print("done")
